## AML 02
***

### Data Preparation

In [1]:
from sklearn.datasets import load_digits
from data_preparation import data_preparation

import numpy as np

In [2]:
digits = load_digits()
print(digits.keys())

dict_keys(['data', 'target', 'frame', 'feature_names', 'target_names', 'images', 'DESCR'])


In [3]:
data = digits['data']
images = digits['images']
target = ['target']
target_names = ['target_names']

In [4]:
data, target = data_preparation(digits, filter=[3, 8])

[3, 8]


In [7]:
data.shape

(357, 1, 64)